# CS5760 — Natural Language Processing  
**Homework 1 — Answers & Code**  

**Student:** _(fill your name here)_  
**Semester:** Fall 2025  
**Course:** University of Central Missouri — CS5760

> This notebook contains solutions and runnable code for:
> - Q1: Regular Expressions
> - Q2: Tokenization (naïve vs tool), MWEs, Reflection
> - Q3: Byte Pair Encoding (manual and code)
> - Q4: Minimum Edit Distance (two cost models + DP table)

## Q1. Regex

We provide patterns and quick demonstrations. Adjust test strings as needed.

In [1]:

import re

test_text = """
ZIP examples: 12345, 12345-6789, 12345 6789, bad: 1234, embeddedX12345Y.
Words: apple, Apple, don't, state-of-the-art, banana.
Numbers: +1, -2, 1,234, 12,345.67, -3.5e-2, +10E+3, 007, 1,2,3 (bad).
Email variants: email, e-mail, e–mail, e mail, E-Mail, NOT: remail, emailed (should not match whole-token).
Interjection: go, goo, gooo!, go?, g, ogoo, gooo, gooo.
Questions: He said "Really?"  Did you ask “why?”)   End like this?  Or maybe?
""".strip()

# (a) U.S. ZIP codes: 12345 OR 12345-6789 OR 12345 6789, only as whole tokens (no embedding)
pattern_zip = re.compile(r"\b\d{5}(?:[-\s]\d{4})?\b")
zips = pattern_zip.findall(test_text)

# (b) Words that do not start with a capital letter (allow internal apostrophes/hyphens)
pattern_noncap = re.compile(r"\b(?![A-Z])[A-Za-z]+(?:['-][A-Za-z]+)*\b")
noncap_words = pattern_noncap.findall(test_text)

# (c) Numbers with optional sign, thousands separators, decimals, and scientific notation
pattern_numbers = re.compile(r"[+-]?(?:\d{1,3}(?:,\d{3})*|\d+)(?:\.\d+)?(?:[eE][+-]?\d+)?")
numbers = pattern_numbers.findall(test_text)

# (d) Variants of “email”: email, e-mail, e–mail (en-dash), e mail (case-insensitive), word-boundary
pattern_email = re.compile(r"(?i)\be(?:[-–\s]?mail)\b")
emails = pattern_email.findall(test_text)

# (e) Interjection: go, goo, gooo… as a word, optional trailing punctuation ! . , ?
pattern_go = re.compile(r"\bgo+[\!\.\,\?]?\b", re.IGNORECASE)
go_matches = pattern_go.findall(test_text)

# (f) Lines that end with a question mark possibly followed only by closing quotes/brackets and spaces
# We'll test per-line:
pattern_question_end = re.compile(r"\?[)\"”’\]\s]*$")
question_lines = []
for line in test_text.splitlines():
    if pattern_question_end.search(line):
        question_lines.append(line)

print("=== Q1 DEMO RESULTS ===")
print("ZIP matches:", zips)
print("Non-capital-start words:", noncap_words[:12], "... (total:", len(noncap_words), ")")
print("Numbers:", numbers)
print("Email variants:", emails)
print("Interjection 'go+' matches:", go_matches)
print("Lines ending like a question (with closing quotes/brackets):", question_lines)


=== Q1 DEMO RESULTS ===
ZIP matches: ['12345', '12345-6789', '12345 6789']
Non-capital-start words: ['examples', 'bad', 'apple', "don't", 'state-of-the-art', 'banana', 'bad', 'variants', 'email', 'e-mail', 'e', 'mail'] ... (total: 35 )
Numbers: ['123', '45', '123', '45', '-678', '9', '123', '45', '678', '9', '123', '4', '123', '45', '+1', '-2', '1,234', '12,345.67', '-3.5e-2', '+10E+3', '007', '1', '2', '3']
Email variants: ['Email', 'email', 'e-mail', 'e–mail', 'e mail', 'E-Mail']
Interjection 'go+' matches: ['go', 'goo', 'gooo', 'go', 'gooo', 'gooo']
Lines ending like a question (with closing quotes/brackets): ['Questions: He said "Really?"  Did you ask “why?”)   End like this?  Or maybe?']


## Q2. Programming: Tokenization (Telugu example)

We use a short 3–4 sentence paragraph in Telugu, then show:
1) naïve space-based tokenization,  
2) a manual correction (punctuation separated; keep morphemes intact for clarity),  
3) a simple regex-based tokenizer as the "tool" output for comparison, and  
4) MWEs and a brief reflection.

In [ ]:

import re
from collections import Counter
from pprint import pprint

paragraph = "రాము పుస్తకం చదువుతున్నాడు. నేను ఆపిల్ తిన్నాను! ఇది చాలా మంచి కథ."

# (1) Naïve space-based tokenization
naive_tokens = paragraph.split()

# (2) Manual correction: separate trailing punctuation (.!?), keep words otherwise intact
def manual_tokenize(text):
    tokens = []
    for tok in text.split():
        m = re.match(r"^(.*?)([\.!\?,;:\"'”’)\]]+)$", tok)
        if m:
            core, punct = m.group(1), m.group(2)
            if core:
                tokens.append(core)
            tokens.extend(list(punct))  # keep each punctuation as its own token
        else:
            tokens.append(tok)
    return tokens

manual_tokens = manual_tokenize(paragraph)

# (3) "Tool" tokenizer: regex-based (Unicode-aware): words or single punctuation marks
tool_tokens = re.findall(r"\w+|[^\w\s]", paragraph, flags=re.UNICODE)

# Compare differences
def compare_lists(a, b):
    max_len = max(len(a), len(b))
    rows = []
    for i in range(max_len):
        rows.append((i, a[i] if i < len(a) else "", b[i] if i < len(b) else ""))
    return rows

print("=== Q2 DEMO RESULTS ===")
print("Paragraph:", paragraph)
print("\nNaïve tokens:")
print(naive_tokens)
print("\nManual tokens:")
print(manual_tokens)
print("\nTool tokens (regex-based):")
print(tool_tokens)

rows_naive_manual = compare_lists(naive_tokens, manual_tokens)
rows_manual_tool = compare_lists(manual_tokens, tool_tokens)

# Show first 20 comparisons for brevity
print("\nNaïve vs Manual (first 20):")
for r in rows_naive_manual[:20]:
    print(r)

print("\nManual vs Tool (first 20):")
for r in rows_manual_tool[:20]:
    print(r)

# (4) MWEs (examples) — reasons they should be treated as a single unit
mwes = [
    ("హైదరాబాద్ నగరం", "Place name; treated as one semantic unit."),
    ("రైల్వే స్టేషన్", "Compound noun frequently used as a fixed phrase."),
    ("చేతులు కాలేయడం", "Idiom; literal tokenization obscures idiomatic meaning.")
]

print("\nMWEs (example rationale):")
for m, why in mwes:
    print(f"- {m}: {why}")


### Q2 — Reflection (5–6 sentences)
Tokenization in Telugu is challenging due to agglutinative morphology: tense, aspect, person, case, and honorific markers attach directly to stems (e.g., *చదువు → చదువుతున్నాడు*). Naïve space splitting leaves punctuation glued to words and ignores suffix boundaries that carry meaning. Compared with English, where spaces correspond fairly well to tokens, Telugu requires either rule-based morphological heuristics or subword methods to capture meaning-bearing units. Multiword expressions like place names and idioms are also problematic because they are syntactically multi-token but semantically single units. Simple Unicode-aware regex tokenizers improve punctuation handling but still miss morpheme boundaries. For robust downstream NLP, subword models or morphological analyzers are preferable to raw whitespace tokenization.

## Q3. Manual BPE and Mini-BPE

We use the toy corpus (with end-of-word `_`) and perform three manual merges, then a small BPE learner.

In [ ]:

from collections import Counter

# Toy corpus words (without underscores), as given
words = "low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new".split()

def add_eow(words):
    # Represent each word as characters plus end-of-word marker '_'
    return [' '.join(list(w)) + ' _' for w in words]

corpus = add_eow(words)

def get_pair_stats(corpus):
    counts = Counter()
    for w in corpus:
        syms = w.split()
        for i in range(len(syms)-1):
            pair = (syms[i], syms[i+1])
            counts[pair] += 1
    return counts

def do_merge(corpus, pair):
    a, b = pair
    pattern = f"{a} {b}"
    repl = f"{a}{b}"
    new_corpus = [w.replace(pattern, repl) for w in corpus]
    return new_corpus, repl

print("=== Q3.1 Manual BPE (First 3 merges) ===")
for step in range(1, 4):
    stats = get_pair_stats(corpus)
    best = max(stats, key=stats.get)
    corpus, new_token = do_merge(corpus, best)
    print(f"Step {step}: merge {best} -> '{new_token}'")
    print("Sample lines:", corpus[:4], "\n")

# After 3 merges, show current vocabulary
vocab = Counter()
for w in corpus:
    for s in w.split():
        vocab[s] += 1

print("Current vocabulary size:", len(vocab))
print("Some tokens:", list(vocab.keys())[:20])


In [ ]:

from collections import Counter

def learn_bpe(words, num_merges=10):
    corpus = [' '.join(list(w)) + ' _' for w in words]
    merges = []
    for i in range(num_merges):
        # count pairs
        counts = Counter()
        for w in corpus:
            syms = w.split()
            for j in range(len(syms)-1):
                counts[(syms[j], syms[j+1])] += 1
        if not counts:
            break
        best = max(counts, key=counts.get)
        merges.append(best)
        # apply merge
        a, b = best
        pattern = f"{a} {b}"
        repl = f"{a}{b}"
        corpus = [w.replace(pattern, repl) for w in corpus]
        print(f"Merge {i+1}: {best}, top count={counts[best]}")
    return merges

def apply_bpe(word, merges):
    # segment a word using learned merges (left-to-right, greedy via replacements on the space-joined representation)
    s = ' '.join(list(word)) + ' _'
    for a, b in merges:
        s = s.replace(f"{a} {b}", f"{a}{b}")
    return s.split()

toy_words = "low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new".split()
merges = learn_bpe(toy_words, num_merges=12)

print("\nLearned merges:", merges)
print("Vocabulary size (approx tokens):", len(set(t for w in toy_words for t in apply_bpe(w, merges))))

# Segment requested words
for w in ["new", "newer", "lowest", "widest", "newestest"]:
    print(w, "→", apply_bpe(w, merges))


### Q3 — Reflection (5–6 sentences)
Subword tokens mitigate the OOV problem by decomposing unseen words into known pieces, allowing the model to handle novel compositions like *newestest* with segments such as `new`, `est`, and `_`. In the toy English corpus, some merges align with meaningful morphemes (e.g., `er_` as the comparative/agentive suffix), while others are purely frequency-driven. As merges increase, frequent stems like `new` and affixes like `er_` or `est_` tend to emerge. However, subword splits may not always respect true morphological boundaries, creating unnatural fragments. For agglutinative languages, many suffixes can be learned as subwords, improving generalization. The balance between vocabulary size and granularity influences both efficiency and linguistic fidelity.

### Q3.3 — BPE on your own paragraph (English example for reproducibility)

We train on a small English paragraph (so this notebook runs offline with consistent output), learn 30 merges, show top 5 merges and 5 longest subword tokens, and segment 5 words (including a rare and an inflected form).

In [ ]:

import re
from collections import Counter

paragraph_en = (
    "Ramu is reading a small story. "
    "This story is about reading habits and readers. "
    "Reading stories helps readers improve skills. "
    "Some readers read slower, others read faster."
)

def train_bpe_from_text(text, num_merges=30):
    words = []
    for w in re.findall(r"\w+", text.lower(), flags=re.UNICODE):
        words.append(w)
    corpus = [' '.join(list(w)) + ' _' for w in words]
    merges = []
    for _ in range(num_merges):
        counts = Counter()
        for w in corpus:
            syms = w.split()
            for i in range(len(syms)-1):
                counts[(syms[i], syms[i+1])] += 1
        if not counts:
            break
        best = max(counts, key=counts.get)
        merges.append(best)
        a, b = best
        pattern = f"{a} {b}"
        repl = f"{a}{b}"
        corpus = [w.replace(pattern, repl) for w in corpus]
    vocab = Counter()
    for w in corpus:
        for s in w.split():
            vocab[s] += 1
    return merges, vocab

def apply_merges(word, merges):
    s = ' '.join(list(word)) + ' _'
    for a, b in merges:
        s = s.replace(f"{a} {b}", f"{a}{b}")
    return s.split()

merges_en, vocab_en = train_bpe_from_text(paragraph_en, num_merges=30)

# Approximate top pairs after training by recomputing pair counts on the merged corpus
def top_pairs_after_training(text, merges, topk=5):
    words = re.findall(r"\w+", text.lower(), flags=re.UNICODE)
    corpus = [' '.join(list(w)) + ' _' for w in words]
    for a,b in merges:
        pattern = f"{a} {b}"
        repl = f"{a}{b}"
        corpus = [w.replace(pattern, repl) for w in corpus]
    counts = Counter()
    for w in corpus:
        syms = w.split()
        for i in range(len(syms)-1):
            counts[(syms[i], syms[i+1])] += 1
    return counts.most_common(topk)

# Five longest subword tokens
longest_tokens = sorted(vocab_en.keys(), key=len, reverse=True)[:5]

print("=== Q3.3 BPE on English paragraph ===")
print("Paragraph:", paragraph_en)
print("\nFive most frequent pairs after training (approx):")
print(top_pairs_after_training(paragraph_en, merges_en, topk=5))

print("\nFive longest subword tokens:")
print(longest_tokens)

# Segment 5 words (include a rare and an inflected form)
for w in ["reading", "readers", "skills", "slower", "unreadable"]:
    print(w, "→", apply_merges(w.lower(), merges_en))


**Brief reflection (5–8 sentences).**  
On this small English paragraph, BPE learned stems like `read` and recurring suffixes like `ing_` and `ers_`. Longer tokens often correspond to frequent subsequences such as entire short words (`is_`, `a_`) or high-frequency bigrams merged repeatedly. Subword tokenization works well for handling derived forms like *readers* and *reading*, and it can also process an unseen word like *unreadable* by combining known pieces (e.g., `un`, `read`, `able_`). Pros: reduces OOVs and captures productive morphology. Cons: merges can split along non-morphemic boundaries and may overfit to the training snippet. For Telugu or other agglutinative languages, BPE can learn common suffixes but might still fragment long, low-frequency words, making interpretability and alignment with true morphology imperfect.

## Q4. Word Pair: Sunday → Saturday

We compute distances under two cost models and produce a DP table excerpt for the (Sub=2) model.

In [ ]:

def min_edit_distance(s1, s2, sub_cost=1, ins_cost=1, del_cost=1):
    n, m = len(s1), len(s2)
    D = [[0]*(m+1) for _ in range(n+1)]
    for i in range(1, n+1):
        D[i][0] = D[i-1][0] + del_cost
    for j in range(1, m+1):
        D[0][j] = D[0][j-1] + ins_cost
    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = 0 if s1[i-1] == s2[j-1] else sub_cost
            D[i][j] = min(
                D[i-1][j] + del_cost,       # deletion
                D[i][j-1] + ins_cost,       # insertion
                D[i-1][j-1] + cost          # substitution / match
            )
    return D

s1 = "SUNDAY"
s2 = "SATURDAY"

# Model A
D_A = min_edit_distance(s1, s2, sub_cost=1, ins_cost=1, del_cost=1)
dist_A = D_A[len(s1)][len(s2)]

# Model B
D_B = min_edit_distance(s1, s2, sub_cost=2, ins_cost=1, del_cost=1)
dist_B = D_B[len(s1)][len(s2)]

print("=== Q4 Distances ===")
print("Model A (sub=1, ins=1, del=1):", dist_A)
print("Model B (sub=2, ins=1, del=1):", dist_B)

# One valid edit sequence (informal) from Sunday -> Saturday
edit_sequence = [
    "SUNDAY",
    "SU NDAY  -> insert 'A' after 'S' → SAUNDAY",
    "SAUNDAY  -> insert 'T' after 'SA' → SATUNDAY",
    "SATUNDAY -> substitute 'N' with 'R' → SATURDAY",
]
print("\nOne valid edit sequence:")
for step in edit_sequence:
    print("-", step)

# DP table excerpt for sub=2: first 3 rows (i=0..3) and first 4 columns (j=0..4)
D = D_B  # sub=2
labels_rows = ["", *s1[:3]]  # "", S, U, N
labels_cols = ["", *s2[:4]]  # "", S, A, T, U

print("\nDP Table (sub=2) — rows i=0..3 (S1), cols j=0..4 (S2):")
for i in range(0, 4):
    row_vals = D[i][:5]
    print(f"i={i:>1} ({labels_rows[i]:>1}) :", row_vals)

print("\nD(3,4) =", D[3][4])
